In [1]:
import pandas as pd
from sqlalchemy import create_engine


# Connect our bd
engine = create_engine('sqlite:///../data/company.db')

Connect to data base

In [2]:
def run_query(query):
    with engine.connect() as connection:
        result = pd.read_sql_query(query, connection)
    return result


# Advances Sub queries 

In [3]:
#! pending
table = "employee"
query1 = f"""
    SELECT * FROM {table}
"""

result1 = run_query(query1)
print("simple query")
result1

simple query


,id,name,last_name,document,department,manager,salary,experience
0,100,David,Ruiz,8325476,CEO,0,5000000,SENIOR
1,101,Ana,García,10234567,Direccion comercial,100,3800000,SEMISENIOR
2,102,Carlos,López,10345678,Direccion Financiera,100,3000000,SEMISENIOR
3,103,María,Rodríguez,10456789,Direccion Marketing,100,4200000,SENIOR
4,104,José,Martínez,10567890,Cartera,102,2600000,JUNIOR
5,105,Laura,Hernández,10678901,Analista,102,3800000,SEMISENIOR
6,106,Pedro,Gómez,10789012,Publicidad,103,3200000,SEMISENIOR
7,107,Marta,Sánchez,10890123,Tesoreria,102,4400000,SENIOR
8,108,Luis,Díaz,10901234,Digital,103,1800000,JUNIOR
9,109,Carmen,Ramírez,11012345,Contador,102,2300000,JUNIOR


Sub query 1

In [4]:
row = "salary"

select_employees = f"""
    SELECT id, 
    last_name, 
    name, 
    salary, 
    RANK() OVER (ORDER BY {row} DESC) AS ranking
    FROM employee
"""
query2 = f"""
    {select_employees}
    ORDER BY ranking
"""

result2 = run_query(query2)
print("Sub query 2")
result2

Sub query 2


,id,last_name,name,salary,ranking
0,100,Ruiz,David,5000000,1
1,107,Sánchez,Marta,4400000,2
2,103,Rodríguez,María,4200000,3
3,101,García,Ana,3800000,4
4,105,Hernández,Laura,3800000,4
5,106,Gómez,Pedro,3200000,6
6,102,López,Carlos,3000000,7
7,110,Pérez,Juan,3000000,7
8,104,Martínez,José,2600000,9
9,109,Ramírez,Carmen,2300000,10


3. Example 3: Show the 5 first rows of a data set
3. Example 3: Show the 5 first rows of a data set

CTE: define temporary queries based on what already exists.


In [5]:
condition = "ranking <= 5"
second_select = f"""
    SELECT 
        id, 
        last_name, 
        name, 
        salary
        FROM ranking_employees     
"""
query3 = f"""
    WITH ranking_employees AS (
        {select_employees}
    )
    {second_select}
    WHERE {condition} 
    ORDER BY ranking
"""
result3 = run_query(query3)
print("Example 3")
result3

Example 3


,id,last_name,name,salary
0,100,Ruiz,David,5000000
1,107,Sánchez,Marta,4400000
2,103,Rodríguez,María,4200000
3,101,García,Ana,3800000
4,105,Hernández,Laura,3800000


In [6]:
query4 = f"""
    WITH ranking_employees AS (
        {select_employees}
    )
    {second_select}
        WHERE ranking <= 5
        ORDER BY ranking DESC """
result4 = run_query(query4)
print("Example 4")
result4

Example 4


,id,last_name,name,salary
0,101,García,Ana,3800000
1,105,Hernández,Laura,3800000
2,103,Rodríguez,María,4200000
3,107,Sánchez,Marta,4400000
4,100,Ruiz,David,5000000


Listar la segunda fila mas alta de los datos consultados

In [7]:
condition = "ranking == 2"
query5 = f"""
    WITH ranking_employees AS (
        {select_employees}
    )
    {second_select}
    WHERE {condition}
    ORDER BY ranking DESC """
result5 = run_query(query5)
print("Example 5")
result5

Example 5


,id,last_name,name,salary
0,107,Sánchez,Marta,4400000


# Median
#! FALTA

### Review the median operation

In [8]:
select_employees_ntile = """
    SELECT 
        id,
        last_name, 
        name, 
        salary, 
        NTILE(2) OVER (ORDER BY salary ASC) AS ntile
    FROM employee
"""

In [9]:
query6 = f"""
    WITH ranking_employees AS (
        {select_employees_ntile}
    )
    SELECT 
        id, 
        last_name, 
        name, 
        salary
        FROM ranking_employees
        WHERE ntile = 1
        ORDER BY salary
        """
result6 = run_query(query6)
print("Example 6")
result6

Example 6


,id,last_name,name,salary
0,108,Díaz,Luis,1800000
1,109,Ramírez,Carmen,2300000
2,104,Martínez,José,2600000
3,102,López,Carlos,3000000
4,110,Pérez,Juan,3000000
5,106,Gómez,Pedro,3200000


## missing the text

In [10]:
select_employees_ntile = """
    SELECT 
        id,
        last_name, 
        name, 
        salary, 
        NTILE(10) OVER (ORDER BY salary ASC) AS ntile
    FROM employee
"""

second_select = """
    SELECT 
        id, 
        last_name, 
        name, 
        salary
        FROM ranking_employees
        WHERE ntile = 5
        ORDER BY salary
"""

In [11]:
query7 = f"""
    WITH ranking_employees AS (
        {select_employees_ntile}
    )
    {second_select}
    """
result7 = run_query(query7)
print("Example 7")
result7

Example 7


,id,last_name,name,salary
0,106,Gómez,Pedro,3200000


8. Example 8: To list between 40% and 70% of data set

In [12]:
second_select = """
    SELECT 
        id, 
        last_name, 
        name, 
        salary
        FROM ranking_employees
        WHERE ntile > 4 AND ntile <= 7
        ORDER BY salary
"""

In [13]:
query8 = f"""
    WITH ranking_employees AS (
        {select_employees_ntile}
    )
    {second_select}    
    """
result8 = run_query(query8)
print("Example 8")
result8

Example 8


,id,last_name,name,salary
0,106,Gómez,Pedro,3200000
1,101,García,Ana,3800000
2,105,Hernández,Laura,3800000



# october 28th - 2024

9. Example 9: Show all register up the ...

In [24]:
select_employee = """
    SELECT last_name,
    name,
    salary
    FROM employee
"""

In [25]:
query9 = f"""
    {select_employee}
    WHERE salary > (SELECT AVG(salary) FROM employee)
"""
result9  = run_query(query9)

# result AVG
query9_2 = "SELECT AVG(salary) FROM employee"
result9_2 = run_query(query9_2)

print("Result AVG: ",result9_2)
print("Select Example 9 with AVG")
result9

Result AVG:      AVG(salary)
0  3.372727e+06
Select Example 9 with AVG


,last_name,name,salary
0,Ruiz,David,5000000
1,García,Ana,3800000
2,Rodríguez,María,4200000
3,Hernández,Laura,3800000
4,Sánchez,Marta,4400000


10. Example 10....

In [28]:
query10 = f"""
    {select_employee}
    WHERE salary < (SELECT AVG(salary) FROM employee)
"""
result10  = run_query(query10)

# result AVG
print("Result AVG: ",result9_2)
print("Select Example 10 with AVG")
result10

Result AVG:      AVG(salary)
0  3.372727e+06
Select Example 10 with AVG


,last_name,name,salary
0,López,Carlos,3000000
1,Martínez,José,2600000
2,Gómez,Pedro,3200000
3,Díaz,Luis,1800000
4,Ramírez,Carmen,2300000
5,Pérez,Juan,3000000


11. Example 11: ....